In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sweetviz as sv
import seaborn as sns
from textblob import TextBlob

In [2]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tktra\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\tktra\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [3]:
test_data = pd.read_csv('Datasets\Religious_text_test.csv')
test_data = test_data.rename(columns={'# foolishness': 'foolishness'})
test_data.head(5)

,foolishness,hath,wholesome,takest,feelings,anger,vaivaswata,matrix,kindled,convict,...,erred,thinkest,modern,reigned,sparingly,visual,thoughts,illumines,attire,explains
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_data = pd.read_csv('Datasets\Religious_text_train.csv')
train_data = train_data.rename(columns={'Unnamed: 0': 'Chapters'})
train_data.head(5)

,Chapters,foolishness,hath,wholesome,takest,feelings,anger,vaivaswata,matrix,kindled,...,erred,thinkest,modern,reigned,sparingly,visual,thoughts,illumines,attire,explains
0,Buddhism_Ch1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Buddhism_Ch2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Buddhism_Ch3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Buddhism_Ch4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Buddhism_Ch5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Training Data Split By Book (with Columns Maintained)

In [5]:
buddhism = train_data[train_data['Chapters'].str.contains('Buddhism')]
tao_te_ching = train_data[train_data['Chapters'].str.contains('TaoTeChing')]
upanishad = train_data[train_data['Chapters'].str.contains('Upanishad')]
yoga_sutra = train_data[train_data['Chapters'].str.contains('YogaSutra')]
book_proverb = train_data[train_data['Chapters'].str.contains('Proverb')]
book_of_ecclesiastes = train_data[train_data['Chapters'].str.contains('Ecclesiastes')]
book_of_eccleasiasticus = train_data[train_data['Chapters'].str.contains('Eccleasiasticus')]
book_of_wisdom = train_data[train_data['Chapters'].str.contains('Wisdom')]


In [6]:
def find_top_words(df:pd.DataFrame, count: int) -> pd.DataFrame:
    """ Given data frame find the {count} most common words (column values)"""
    df = df.sum(axis=0).drop('Chapters').squeeze()
    df = pd.DataFrame({'count': df})
    return df.sort_values(by='count', ascending=False).head(count)

#### Training Data Split By Book Aggregated

In [7]:
book_names = ['Buddhism', 'TaoTeChing', 'Upanishad', 'YogaSutra', 'Proverb', 'Ecclesiastes', 'Eccleasiasticus', 'Wisdom']

In [31]:
pat = '|'.join(book_names)
by_book = train_data['Chapters'].str.extract('(' + pat + ')', expand = False)
books = train_data.groupby(by_book).sum().reset_index().rename(columns={'Chapters':'Books'}).set_index('Books')
books.head(5)

,foolishness,hath,wholesome,takest,feelings,anger,vaivaswata,matrix,kindled,convict,...,erred,thinkest,modern,reigned,sparingly,visual,thoughts,illumines,attire,explains
Books,,,,,,,,,,,,,,,,,,,,,
Buddhism,0,0,0,0,19,0,0,0,0,0,...,0,0,0,0,0,0,9,0,0,0
Eccleasiasticus,0,189,3,1,0,14,0,0,3,0,...,0,0,0,1,1,0,7,0,2,0
Ecclesiastes,0,46,0,0,0,5,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
Proverb,2,65,0,0,0,11,0,0,0,0,...,0,0,0,0,0,0,8,0,1,0
TaoTeChing,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
def get_word_pos_pairs(df:pd.DataFrame) -> list:
    """ Given dataframe, returns all words (columns) and associated part of speech"""
    all_words = TextBlob(" ".join(df.columns))
    return all_words.tags

In [57]:
def get_pos_tag_list(df:pd.DataFrame) -> list:
    """ Returns list of all unique parts of speech found in all words (columns) of a dataframe"""
    word_tags = get_word_pos_pairs(df)
    return list(set([tag[1] for tag in word_tags]))

In [58]:
def get_data_for_pos(df:pd.DataFrame, pos:str) -> pd.DataFrame:
    """Given a dataframe and part of speech, subsets dataframe to only include columns matching the pos"""
    word_tags = get_word_pos_pairs(df)
    chosen_pos = [tag[0] for tag in word_tags if tag[1] == pos]
    return df.loc[:,df.columns.isin(chosen_pos)]

In [59]:
get_data_for_pos(books, 'VB') # Example run retreiving all basic verbs

,build,coward,handle,think,keep,inculcate,disclose,feed,balance,rehearse,...,influence,gaineth,tone,tank,dathan,impose,realms,try,godlike,roar
Books,,,,,,,,,,,,,,,,,,,,,
Buddhism,0,0,0,6,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
Eccleasiasticus,3,1,0,7,27,0,5,2,3,1,...,0,0,0,0,1,0,0,4,0,1
Ecclesiastes,1,0,0,1,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Proverb,1,0,0,5,24,0,0,0,2,0,...,0,1,0,0,0,0,0,0,0,0
TaoTeChing,0,0,0,4,16,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
Upanishad,0,0,1,11,1,0,0,0,0,0,...,1,0,0,0,0,0,5,2,1,0
Wisdom,1,0,1,4,5,0,0,1,1,0,...,0,0,0,0,0,0,0,2,0,0
YogaSutra,5,0,2,7,4,0,0,0,1,0,...,2,0,1,1,0,1,4,4,0,0
